# Téléchargement des libs

In [74]:
!pip install pyspark

# Import des libraries

In [75]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, lower, regexp_replace, regexp_extract, trim, split, expr
from pyspark.sql.types import IntegerType

# Lecture des fichiers CSV

In [130]:
# Créer une session Spark
spark = SparkSession.builder.appName("OlympicDataProcessing").getOrCreate()

# Charger les fichiers CSV
athletes = spark.read.csv('files/olympic_athletes.csv', header=True, inferSchema=True)
results = spark.read.csv('files/olympic_results.csv', header=True, inferSchema=True)
medals = spark.read.csv('files/olympic_medals.csv', header=True, inferSchema=True)
hosts = spark.read.csv('files/olympic_hosts.csv', header=True, inferSchema=True)

# Affichage des 5 premières données de chaque CSV

In [133]:
# Affichez les premières lignes des DataFrames pour une inspection rapide
athletes.show(5)
results.show(5)
medals.show(5)
hosts.show(5)

+--------------------+--------------------+--------------------+----------------+------------------+--------------+----+
|         athlete_url|   athlete_full_name|games_participations|      first_game|athlete_year_birth|athlete_medals| bio|
+--------------------+--------------------+--------------------+----------------+------------------+--------------+----+
|https://olympics....|Cooper WOODS-TOPA...|                   1|    Beijing 2022|            2000.0|          NULL|NULL|
|https://olympics....|      Felix ELOFSSON|                   2|PyeongChang 2018|            1995.0|          NULL|NULL|
|https://olympics....|       Dylan WALCZYK|                   1|    Beijing 2022|            1993.0|          NULL|NULL|
|https://olympics....|       Olli PENTTALA|                   1|    Beijing 2022|            1995.0|          NULL|NULL|
|https://olympics....|    Dmitriy REIKHERD|                   1|    Beijing 2022|            1989.0|          NULL|NULL|
+--------------------+----------

# Afficher le nombre de lignes par CSV

In [78]:
# Affichez le nombre de lignes des DataFrames
print("Nombre de ligne athletes:", athletes.count())
print("Nombre de ligne hosts:", hosts.count())
print("Nombre de ligne medals:", medals.count())
print("Nombre de ligne results:", results.count())

Nombre de ligne athletes: 189960
Nombre de ligne hosts: 53
Nombre de ligne medals: 21697
Nombre de ligne results: 162804


# Afficher les types de données

In [79]:
# Afficher les schémas des DataFrames
print("Schéma de athletes:")
athletes.printSchema()

print("Schéma de results:")
results.printSchema()

print("Schéma de medals:")
medals.printSchema()

print("Schéma de hosts:")
hosts.printSchema()


Schéma de athletes:
root
 |-- athlete_url: string (nullable = true)
 |-- athlete_full_name: string (nullable = true)
 |-- games_participations: string (nullable = true)
 |-- first_game: string (nullable = true)
 |-- athlete_year_birth: string (nullable = true)
 |-- athlete_medals: string (nullable = true)
 |-- bio: string (nullable = true)

Schéma de results:
root
 |-- discipline_title: string (nullable = true)
 |-- event_title: string (nullable = true)
 |-- slug_game: string (nullable = true)
 |-- participant_type: string (nullable = true)
 |-- medal_type: string (nullable = true)
 |-- athletes: string (nullable = true)
 |-- rank_equal: string (nullable = true)
 |-- rank_position: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country_3_letter_code: string (nullable = true)
 |-- athlete_url: string (nullable = true)
 |-- athlete_full_name: string (nullable = true)
 |-- value_unit: string (nullable = true)
 |-- valu

# Afficher les données manquantes

In [80]:
# Vérifiez les valeurs manquantes
athletes.select([count(when(col(c).isNull(), c)).alias(c) for c in athletes.columns]).show()
results.select([count(when(col(c).isNull(), c)).alias(c) for c in results.columns]).show()
medals.select([count(when(col(c).isNull(), c)).alias(c) for c in medals.columns]).show()
hosts.select([count(when(col(c).isNull(), c)).alias(c) for c in hosts.columns]).show()

+-----------+-----------------+--------------------+----------+------------------+--------------+------+
|athlete_url|athlete_full_name|games_participations|first_game|athlete_year_birth|athlete_medals|   bio|
+-----------+-----------------+--------------------+----------+------------------+--------------+------+
|          0|            94041|               96547|     99234|            104133|        164540|166953|
+-----------+-----------------+--------------------+----------+------------------+--------------+------+

+----------------+-----------+---------+----------------+----------+--------+----------+-------------+------------+------------+---------------------+-----------+-----------------+----------+----------+
|discipline_title|event_title|slug_game|participant_type|medal_type|athletes|rank_equal|rank_position|country_name|country_code|country_3_letter_code|athlete_url|athlete_full_name|value_unit|value_type|
+----------------+-----------+---------+----------------+----------+

# Afficher les données dupliquées

In [81]:
# Afficher le nombre de lignes dupliquées dans chaque DataFrame
print("Nombre de lignes dupliquées dans athletes:", athletes.count() - athletes.dropDuplicates().count())
print("Nombre de lignes dupliquées dans results:", results.count() - results.dropDuplicates().count())
print("Nombre de lignes dupliquées dans medals:", medals.count() - medals.dropDuplicates().count())
print("Nombre de lignes dupliquées dans hosts:", hosts.count() - hosts.dropDuplicates().count())

Nombre de lignes dupliquées dans athletes: 78182
Nombre de lignes dupliquées dans results: 116
Nombre de lignes dupliquées dans medals: 0
Nombre de lignes dupliquées dans hosts: 0


# Afficher le nombre de données manquantes pour chaque colonne

In [82]:
# Afficher le nombre de valeurs manquantes dans chaque colonne pour chaque DataFrame
print("Nombre de valeurs manquantes dans athletes:")
athletes.select([count(when(col(c).isNull(), c)).alias(c) for c in athletes.columns]).show()

print("Nombre de valeurs manquantes dans results:")
results.select([count(when(col(c).isNull(), c)).alias(c) for c in results.columns]).show()

print("Nombre de valeurs manquantes dans medals:")
medals.select([count(when(col(c).isNull(), c)).alias(c) for c in medals.columns]).show()

print("Nombre de valeurs manquantes dans hosts:")
hosts.select([count(when(col(c).isNull(), c)).alias(c) for c in hosts.columns]).show()

Nombre de valeurs manquantes dans athletes:
+-----------+-----------------+--------------------+----------+------------------+--------------+------+
|athlete_url|athlete_full_name|games_participations|first_game|athlete_year_birth|athlete_medals|   bio|
+-----------+-----------------+--------------------+----------+------------------+--------------+------+
|          0|            94041|               96547|     99234|            104133|        164540|166953|
+-----------+-----------------+--------------------+----------+------------------+--------------+------+

Nombre de valeurs manquantes dans results:
+----------------+-----------+---------+----------------+----------+--------+----------+-------------+------------+------------+---------------------+-----------+-----------------+----------+----------+
|discipline_title|event_title|slug_game|participant_type|medal_type|athletes|rank_equal|rank_position|country_name|country_code|country_3_letter_code|athlete_url|athlete_full_name|valu

# Suppression des colonnes

In [154]:
# Supprimer des colonnes dans chaque DataFrame CSV

# Pour athletes.csv
# athletes = athletes.drop("")

# Pour results.csv
df_results = results.drop('Unnamed: 0','athlete_url')

# Pour medals.csv
df_medals = medals.drop('Unnamed: 0','participant_title','athlete_url')

# Pour hosts.csv
df_hosts = hosts.drop("index")

In [155]:
# Afficher les schémas des DataFrames
#print("Schéma de athltes:")
#athletes.printSchema()

print("Schéma de results:")
df_results.printSchema()

print("Schéma de medals:")
df_medals.printSchema()

print("Schéma de hosts:")
df_hosts.printSchema()

Schéma de results:
root
 |-- discipline_title: string (nullable = true)
 |-- event_title: string (nullable = true)
 |-- slug_game: string (nullable = true)
 |-- participant_type: string (nullable = true)
 |-- medal_type: string (nullable = true)
 |-- athletes: string (nullable = true)
 |-- rank_equal: string (nullable = true)
 |-- rank_position: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country_3_letter_code: string (nullable = true)
 |-- athlete_full_name: string (nullable = true)
 |-- value_unit: string (nullable = true)
 |-- value_type: string (nullable = true)

Schéma de medals:
root
 |-- discipline_title: string (nullable = true)
 |-- slug_game: string (nullable = true)
 |-- event_title: string (nullable = true)
 |-- medal_type: string (nullable = true)
 |-- participant_type: string (nullable = true)
 |-- athlete_full_name: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- country_c

# Transformation des champs pour chaque CSV

In [157]:
# Pour athletes.csv
df_athletes = athletes.withColumn("athlete_year_birth", athletes["athlete_year_birth"].cast(IntegerType()))

# Pour results

# Pour medal.csv

# Pour host.csv
df_hosts = hosts.withColumnRenamed("game_slug", "slug_game")

print("Schéma de athletes:")
df_athletes.printSchema()

#print("Schéma de results:")

#print("Schéma de medals:")

print("Schéma de hosts:")
df_hosts.printSchema()

Schéma de athletes:
root
 |-- athlete_url: string (nullable = true)
 |-- athlete_full_name: string (nullable = true)
 |-- games_participations: string (nullable = true)
 |-- first_game: string (nullable = true)
 |-- athlete_year_birth: integer (nullable = true)
 |-- athlete_medals: string (nullable = true)
 |-- bio: string (nullable = true)

Schéma de hosts:
root
 |-- slug_game: string (nullable = true)
 |-- game_end_date: timestamp (nullable = true)
 |-- game_start_date: timestamp (nullable = true)
 |-- game_location: string (nullable = true)
 |-- game_name: string (nullable = true)
 |-- game_season: string (nullable = true)
 |-- game_year: integer (nullable = true)



# Transformation des valeurs pour chaque CSV

Pour athletes

In [158]:
# Pour athletes.csv
# Transformer la colonne "first_game" en minuscule et remplacer les espaces par des tirets
df_athletes = df_athletes.withColumn("first_game", regexp_replace(lower(df_athletes["first_game"]), " ", "-"))

# Nettoyer et transformer la colonne "athlete_medals"
df_athletes = df_athletes.withColumn("athlete_medals", trim(regexp_replace(df_athletes["athlete_medals"], "\n", "")))
df_athletes = df_athletes.withColumn("athlete_medals", regexp_replace(df_athletes["athlete_medals"], "(G|S|B)", "$1,"))
df_athletes = df_athletes.withColumn("athlete_medals", regexp_replace(df_athletes["athlete_medals"], ",$", ""))

# Nettoyer et transformer la colonne "bio"
df_athletes = df_athletes.withColumn("bio", trim(regexp_replace(df_athletes["bio"], "\n", "")))

# Afficher le schéma des données après conversion pour chaque DataFrame
df_athletes.printSchema()
# Afficher les premières lignes après conversion
df_athletes.show()

root
 |-- athlete_url: string (nullable = true)
 |-- athlete_full_name: string (nullable = true)
 |-- games_participations: string (nullable = true)
 |-- first_game: string (nullable = true)
 |-- athlete_year_birth: integer (nullable = true)
 |-- athlete_medals: string (nullable = true)
 |-- bio: string (nullable = true)

+--------------------+--------------------+--------------------+----------------+------------------+--------------+----+
|         athlete_url|   athlete_full_name|games_participations|      first_game|athlete_year_birth|athlete_medals| bio|
+--------------------+--------------------+--------------------+----------------+------------------+--------------+----+
|https://olympics....|Cooper WOODS-TOPA...|                   1|    beijing-2022|              2000|          NULL|NULL|
|https://olympics....|      Felix ELOFSSON|                   2|pyeongchang-2018|              1995|          NULL|NULL|
|https://olympics....|       Dylan WALCZYK|                   1|    bei

# Traitement

In [162]:
jo_types = df_hosts[['slug_game','game_season', 'game_year']]
hosts_results_merge = df_hosts.join(results, df_hosts['slug_game'] == df_results['slug_game'])
hosts_results_merge.show(5)

+------------+-------------------+-------------------+-------------+------------+-----------+---------+----------------+-------------+------------+----------------+----------+--------------------+----------+-------------+-------------+------------+---------------------+-----------------+----------+----------+
|   slug_game|      game_end_date|    game_start_date|game_location|   game_name|game_season|game_year|discipline_title|  event_title|   slug_game|participant_type|medal_type|            athletes|rank_equal|rank_position| country_name|country_code|country_3_letter_code|athlete_full_name|value_unit|value_type|
+------------+-------------------+-------------------+-------------+------------+-----------+---------+----------------+-------------+------------+----------------+----------+--------------------+----------+-------------+-------------+------------+---------------------+-----------------+----------+----------+
|beijing-2022|2022-02-20 12:00:00|2022-02-04 15:00:00|        China